# Setup the Agent Development Kit

<img src="img/agent_development_kit_logo.png" align="left" style="height: 100px;">Agent Development Kit (ADK) is a flexible and modular framework for developing and deploying AI agents. While optimized for Gemini and the Google ecosystem, ADK is model-agnostic, deployment-agnostic, and is built for compatibility with other frameworks. ADK was designed to make agent development feel more like software development, to make it easier for developers to create, deploy, and orchestrate agentic architectures that range from simple tasks to complex workflows.

## Basic Setup

### Define Notebook Variables

Update the variables below to match your environment. You will be prompted for the AlloyDB password you chose then you provisioned the environment with Terraform.

In [ ]:
# Project variables
project_id = "your-project"
region = "your-region"
vpc = "demo-vpc"
gcs_bucket_name = f"project-files-{project_id}"

In [ ]:
# Set env variable to suppress annoying system warnings when running shell commands
%env GRPC_ENABLE_FORK_SUPPORT=1

### Connect to your Google Cloud Project

In [ ]:
# Configure gcloud.
!gcloud config set project {project_id}

### Configure Logging

In [ ]:
import logging
import sys

# Configure the root logger to output messages with ERROR level
logging.basicConfig(level=logging.ERROR, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Install Dependencies

In [ ]:
! pip install --quiet toolbox-core==0.2.1 \
                      google-adk==1.5.0 \
                      google-genai==1.23.0


## Build an Agent with MCP Toolbox Tools

### Get the Toolbox Endpoint

In [ ]:
import json

toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(toolbox_url)

### Get Auth Token

We required authenticated invocations of the Cloud Run service, so we first need to grab an auth token to use with our ToolboxClient. 

In [ ]:
import urllib

import google.auth.transport.requests
import google.oauth2.id_token


def get_auth_token(endpoint):
    # Cloud Run uses your service's hostname as the `audience` value
    # audience = 'https://my-cloud-run-service.run.app/'
    # For Cloud Run, `endpoint` is the URL (hostname + path) receiving the request
    # endpoint = 'https://my-cloud-run-service.run.app/my/awesome/url'
    
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, endpoint)

    return id_token

### Define Agent

In [ ]:
# Reference: https://googleapis.github.io/genai-toolbox/getting-started/local_quickstart/

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google import genai
from google.genai import types
from toolbox_core import ToolboxSyncClient

import asyncio
import os

# Use GOOGLE_API_KEY or Vertex AI
# os.environ['GOOGLE_API_KEY'] = 'your-api-key'

os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'
os.environ['GOOGLE_CLOUD_PROJECT']      = project_id
os.environ['GOOGLE_CLOUD_LOCATION']     = region

# Get auth token to invoke Toolbox on Cloud Run
auth_token = get_auth_token(toolbox_url)

async def invoke_agent(query: str):
  with ToolboxSyncClient(
        toolbox_url,
        client_headers={"Authorization": f"Bearer {auth_token}"}
  ) as toolbox_client:

      prompt = """
        You're a helpful financial assistant. You handle fraud detection tasks, transaction lookups,
        account details, loan payments, suspicious activity reports, and other tasks related to 
        financial data. Always provide a customer_id when using tools to lookup information.
      """

      app_name = 'finance_agent'
    
      user_id = '123'

      root_agent = Agent(
          model='gemini-2.5-flash',
          name=app_name,
          description='A helpful AI assistant.',
          instruction=prompt,
          tools=toolbox_client.load_toolset("finance-toolset"),
      )

      session_service = InMemorySessionService()
      artifacts_service = InMemoryArtifactService()
      session = await session_service.create_session(
          state={}, app_name=app_name, user_id=user_id
      )
      runner = Runner(
          app_name=app_name,
          agent=root_agent,
          artifact_service=artifacts_service,
          session_service=session_service,
      )

      content = types.Content(role='user', parts=[types.Part(text=query)])
      events = runner.run(session_id=session.id,
                          user_id=user_id, new_message=content)

      responses = (
        part.text
        for event in events
        for part in event.content.parts
        if part.text is not None
      )

      for text in responses:
        print(text)


### Manually Invoke the Agent

In [ ]:
prompts = [
    "What are the latest trasactions for customer_id 11?",
    "Tell me about the recent account transfers made by customer_id 8 and flag any suspicious activity.",
    "Is anything strange about the latest tranactions made by customer_id 3?",
]

for p in prompts:
    print(f"User prompt: {p}\n")
    print("Agent response:")
    response = await invoke_agent(p)
    print("\n\n")
    

## Inspect Agent Processing

The examples above show that the Agent has access to our databases via MCP Toolbox, because the output contains data the Agent wouldn't otherwise know. But what if we want to debug an Agent's logic or understand the specific tool calls that were made? When developing locally, we can set the log level to info to see detailed, step-by-step Agent execution and tool call details. 

### Set log level to INFO

In [ ]:
import logging
import sys

# Configure the root logger to output messages with INFO level or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Examine Detailed Output

In [ ]:
prompt = "How many transfers has customer_id 462 made recently? Is there anything strange about the transfers?"
await invoke_agent(prompt)